In [ ]:
!wget https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/Project_CodeNet.tar.gz
!tar -zxvf /content/Project_CodeNet.tar.gz --wildcards "*.c"
!pip install sctokenizer

--2023-01-09 14:51:09--  https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/Project_CodeNet.tar.gz
Resolving dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)... 23.214.66.165, 2600:141b:f000:e08a::d2d, 2600:141b:f000:e086::d2d
Connecting to dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)|23.214.66.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8343137473 (7.8G) [application/x-gtar]
Saving to: ‘Project_CodeNet.tar.gz’

Project_CodeNet.tar 100%[===================>]   7.77G   105MB/s    in 3m 32s  

2023-01-09 14:54:42 (37.5 MB/s) - ‘Project_CodeNet.tar.gz’ saved [8343137473/8343137473]



In [ ]:
import pandas as pd
import sctokenizer
import pickle

In [ ]:
submissions=pd.read_csv('/content/drive/MyDrive/c_train/data/CodeNetSubmissions.csv', sep='\t', encoding='utf-8')
submissions=submissions.sort_values(
    ['problem_id', 'code_size', 'memory', 'cpu_time'], ascending=[True, True, True, True]
    ).drop_duplicates(subset=['problem_id'])[['problem_id', 'submission_id']]

In [ ]:
path=f'/content/Project_CodeNet/data/{submissions.problem_id.values[2]}/C/{submissions.submission_id.values[2]}.c'
for line in open(path, 'r'):
    print(line)

main(a,b){for(;~scanf("%d%d",&a,&b);puts(&a))a=49+log10(a+b);}


In [ ]:
def tokenize(path):
  stdlibNames='abort\n  abs\n  acos\n  asctime\n  asin\n  assert\n  atan\n  atan2\n  atexit\n  atof\n  atoi\n  atol\n  bsearch\n  calloc\n  ceil\n  clearerr\n  clock\n  cos\n  cosh\n  ctime\n  difftime\n  div\n  exit\n  exp\n  fabs\n  fclose\n  feof\n  ferror\n  fflush\n  fgetc\n  fgetpos\n  fgets\n  floor\n  fmod\n  fopen\n  fprintf\n  fputc\n  fputs\n  fread\n  free\n  freopen\n  frexp\n  fscanf\n  fseek\n  fsetpos\n  ftell\n  fwrite\n  getc\n  getchar\n  getenv\n  gets\n  gmtime\n  isalnum\n  isalpha\n  iscntrl\n  isdigit\n  isgraph\n  islower\n  isprint\n  ispunct\n  isspace\n  isupper\n  isxdigit\n  labs\n  ldexp\n  ldiv\n  localeconv\n  localtime\n  log\n  log10\n  longjmp\n  malloc\n  mblen\n  mbstowcs\n  mbtowc\n  memchr\n  memcmp\n  memcpy\n  memmove\n  memset\n  mktime\n  modf\n  perror\n  pow\n  printf\n  putc\n  putchar\n  puts\n  qsort\n  raise\n  rand\n  realloc\n  remove\n  rename\n  rewind\n  scanf\n  setbuf\n  setjmp\n  setlocale\n  setvbuf\n  signal\n  sin\n  sinh\n  sprintf\n  sqrt\n  srand\n  sscanf\n  strcat\n  strchr\n  strcmp\n  strcoll\n  strcpy\n  strcspn\n  strerror\n  strftime\n  strlen\n  strncat\n  strncmp\n  strncpy\n  strpbrk\n  strrchr\n  strspn\n  strstr\n  strtod\n  strtok\n  strtol\n  strtoll\n  strtoul\n  strxfrm\n  system\n  tan\n  tanh\n  time\n  tmpfile\n  tmpnam\n  tolower\n  toupper\n  ungetc\n  va_arg\n  va_end\n  va_start\n  vfprintf\n  vprintf\n  vsprintf\n  wcstombs\n  wctomb'
  stdlibNames=stdlibNames.split('\n  ')

  tokens = sctokenizer.tokenize_file(filepath=path, lang='c')
  temp=set([id.token_value for id in tokens if id.token_type.name=='IDENTIFIER'])
  temp=[id for id in temp if id not in stdlibNames]
  dictionary = dict(zip(temp, [f'id{i}' for i in range(len(temp))]))
  sample=[]
  for token in tokens:
    if token.token_type.name!='IDENTIFIER':
      sample.append(token.token_value)
    elif token.token_value in temp:
      sample.append(dictionary[token.token_value])
    else:
      sample.append(token.token_value)
  return ' '.join(sample)

data=[]
for i in range(submissions.problem_id.values.size):
  data.append(tokenize(f'/content/Project_CodeNet/data/{submissions.problem_id.values[i]}/C/{submissions.submission_id.values[i]}.c'))

with open('/content/drive/MyDrive/c_train/data/problem_dict.pkl', 'rb') as f:
    problem_dict = pickle.load(f)

p_keys=[]
for description in submissions.problem_id.values:
  try:
    p_keys.append(problem_dict[description+'.html'])
  except:
    p_keys.append('')

In [ ]:
df = pd.DataFrame({'nl': p_keys, 'code': data})
df = df[df.nl != '']
df

,nl,code
0,Write a program which prints multiplication ta...,"id0 ( ) { system ( "" /tmp/Z "" ) ; }"
1,There is a data which provides heights in mete...,"id0 , id1 [ ] ; id2 ( int * id4 ) { id0 = * id..."
2,Write a program which computes the digit numbe...,"id1 ( id0 , id2 ) { for ( ; ~ scanf ( "" %d%d ""..."
3,Write a program which judges wheather given le...,"id4 , id2 , id0 ; id3 ( id1 ) { for ( ; ~ scan..."
4,Write a program which solve a simultaneous equ...,"float id0 , id7 , id6 , id5 , id3 , id1 ; id4 ..."
...,...,...
3260,Snuke is having a barbeque party At the party ...,"# include < id4 > int id5 ( ) { int id6 , id2 ..."
3261,Snuke is conducting an optical experiment usin...,"long id5 , id0 , id1 , id3 ; id4 ( ) { for ( s..."
3262,Given an undirected tree let the distance betw...,# include < id5 > # include < id8 > # define i...
3263,Snuke got a present from his mother on his bir...,# include < id3 > # include < id6 > void id4 (...


In [ ]:
df.to_csv('/content/drive/MyDrive/c_train/druga_proba.csv', sep='\t', encoding='utf-8')

In [ ]:
def tokenize(path):
  tokens = sctokenizer.tokenize_file(filepath=path, lang='c')
  sample=[]
  for token in tokens:
    sample.append(token.token_value)
  return ' '.join(sample)

data=[]
for i in range(submissions.problem_id.values.size):
  data.append(tokenize(f'/content/Project_CodeNet/data/{submissions.problem_id.values[i]}/C/{submissions.submission_id.values[i]}.c'))

with open('/content/drive/MyDrive/c_train/data/problem_dict.pkl', 'rb') as f:
    problem_dict = pickle.load(f)

p_keys=[]
for description in submissions.problem_id.values:
  try:
    p_keys.append(problem_dict[description+'.html'])
  except:
    p_keys.append('')

In [ ]:
df = pd.DataFrame({'nl': p_keys, 'code': data})
df = df[df.nl != '']
df

,nl,code
0,Write a program which prints multiplication ta...,"main ( ) { system ( "" /tmp/Z "" ) ; }"
1,There is a data which provides heights in mete...,"d , a [ ] ; c ( int * z ) { d = * z - * 1 [ & ..."
2,Write a program which computes the digit numbe...,"main ( a , b ) { for ( ; ~ scanf ( "" %d%d "" , ..."
3,Write a program which judges wheather given le...,"m , s , a ; main ( i ) { for ( ; ~ scanf ( "" %..."
4,Write a program which solve a simultaneous equ...,"float d , b , f , e , c , a ; main ( X ) { ~ s..."
...,...,...
3260,Snuke is having a barbeque party At the party ...,"# include < stdio.h > int main ( ) { int n , a..."
3261,Snuke is conducting an optical experiment usin...,"long n , x , s , t ; main ( ) { for ( scanf ( ..."
3262,Given an undirected tree let the distance betw...,# include < stdio.h > # include < stdlib.h > #...
3263,Snuke got a present from his mother on his bir...,# include < stdio.h > # include < stdlib.h > v...


In [ ]:
df.to_csv('/content/drive/MyDrive/c_train/trzecia_proba.csv', sep='\t', encoding='utf-8')